In [ ]:
# 참고 사이트 : https://www.kaggle.com/code/alincijov/nlp-starter-continuous-bag-of-words-cbow
# 참고 사이트 : https://towardsdatascience.com/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314

# word embedding : 단어를 유의미한 수치의 벡터로 만들기 위한방법
# one hot encoding 등 여러방법이 있다 ...

# one hot vector 단점
# 1. 단어수 늘어남에 따라 차원수 늘어남
# 2. application r과 밀접하게 결합되어 // 다른 모델로 전이학습
# 혹은 단어의 추가/삭제가 용이하지 않다.
# 3. embedding 의 목적은 단어의 문맥적 의미를 파악하는 것이나, 
# 단어와 단어사이의 문맥적 관계를 파악할 방법이 없다. 

# CBOW : 중심단어 기준으로 중간단어 예측하는거임

# CBOW architecture
# - the word embeddings as inputs (idx)
# - the linear model as the hidden layer
# - the log_softmax as the output

In [ ]:
# CBOW
# 문맥으로 부터 중간단어예측
# 인풋단어들에 대해서 same target words 다.
# skip-gram 보다 빠름, 빈번한 단어들에 대해서는 조금더 정확함.

# Skip-gram
# 단어로부터 문맥예측
# fake task 는 이 네트워크의 input과 output 에 관심이 있는것이 
# 아니라, hidden layer 의 weight 와 word vector 에만 관심이있다.
# input 단어의 접근성은 상관없다 거리 상관 x 
# 작은 양에 대해서 훈련시 훈련잘됨

In [ ]:
sentences = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [ ]:
import re
import torch
import numpy as np

# remove special characters and replace with ' '
# [] => 나열된 문자 혹은 범위에 해당하는 문자

# 특수문자 제거
sentences = re.sub('[^A-Za-z0-9]+', ' ', sentences)

# remove 1 letter word 첫부분 한단어,끝부분 한단어, 중간 한단어짜리 지움
# ?: non-capturing group -> 대충 매칭되도 무시하겠다 이런뜻
sentences =  re.sub(r'(?:^| )\w(?:$| )', ' ', sentences).strip()

# lower all characters
sentences = sentences.lower()

In [ ]:
# vocabulary 변환
words = sentences.split()
vocab = set(words)


In [ ]:
vocab_size = len(vocab)
embed_dim = 10

# sliding window 의 한쪽 면이라고 생각하는게 편한거 같다. 
context_size = 2
print(f"vocab size : {vocab_size}")

In [ ]:
# implementation
word_to_idx = {word: i for i,word in enumerate(vocab)}
idx_to_word = {i:word for i,word in enumerate(vocab)}

In [ ]:
# words -> 정제된 문장
# voca -> 단어 중복제거 
# embde_dim -> 차원 변환 결과
 
 
# data - [(context), target]

data = []
for i in range(2, len(words)-2):
    context = [words[i-2], words[i-1], words[i+1], words[i+2]]
    target = words[i]
    data.append((context, target))

print(f"length of data : {len(data)}")


In [ ]:
# 여기서 부터 pytorch 로 바꿔보면 괜찮을듯 싶다. 

# Embeddings 
embeddings =  np.random.random_sample((vocab_size, embed_dim))
print(f"embedding shape : {embeddings.shape}")
# 43개의 input 10개의 output

In [ ]:
# 행렬곱 시키는거
def linear(m, theta):
    w = theta
    return m.dot(w)

# 결과값 softmax 시키는 것
def log_softmax(x):
    e_x = np.exp(x - np.max(x))
    return np.log(e_x / e_x.sum())

# logs => softmax log 값 취한거
def NLLLoss(logs, targets):
    out = logs[range(len(targets)), targets]
    return -out.sum()/len(out)

# 
def log_softmax_crossentropy_with_logits(logits,target):
    
    out = np.zeros_like(logits)
    out[np.arange(len(logits)),target] = 1
    
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    
    return (- out + softmax) / logits.shape[0]

In [ ]:
# 원핫 벡터를 만드는 대신 숫자로 바로 만들어버려서 
# import numpy as np
# theta = np.random.uniform(-1, 1, (2 * context_size * embed_dim, vocab_size))
# context_idxs = [1,2,3,4]

# context_idxs.shape => (4,1) <- window 갯수 4개라서 그럼

# @내 생각은 () @(10,43) 
# theta.shape => (40,43) <- 
# embeddings => (43,10)

# 
def forward(context_idxs, theta):

    m = embeddings[context_idxs].reshape(1, -1)
    n = linear(m, theta)
    o = log_softmax(n)

    # o.shape => (1, 43)
    return m, n, o
# forward(context_idxs, theta)

In [ ]:
def backward(preds, theta, target_idxs):
    m, n, o = preds
    # m => 1 번째 결과값
    # n => 2 번쨰 결과값
    # o => softmax 에다가 log 취한거
    dlog = log_softmax_crossentropy_with_logits(n, target_idxs)
    dw = m.T.dot(dlog)
                
    return dw

In [ ]:
def optimize(theta, grad, lr=0.03):
    theta -= grad * lr
    return theta

In [ ]:
epoch_losses = {}
# 균등분포로 np.random.uniform 뽑아냄
theta = np.random.uniform(-1, 1, (2 * context_size * embed_dim, vocab_size))
for epoch in range(80):

    losses =  []

    for context, target in data:
        context_idxs = np.array([word_to_idx[w] for w in context])
        
        preds = forward(context_idxs, theta)
        
        target_idxs = np.array([word_to_idx[target]])
        loss = NLLLoss(preds[-1], target_idxs)
        
        losses.append(loss)

        grad = backward(preds, theta, target_idxs)
        theta = optimize(theta, grad, lr=0.03)
        
     
    epoch_losses[epoch] = losses

In [ ]:
import matplotlib.pyplot as plt

ix = np.arange(0,80)

fig = plt.figure()
fig.suptitle('Epoch/Losses', fontsize=20)
plt.plot(ix,[epoch_losses[i][0] for i in ix])
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Losses', fontsize=12)

In [ ]:
def predict(words):
    context_idxs = np.array([word_to_idx[w] for w in words])
    preds = forward(context_idxs, theta)
    word = idx_to_word[np.argmax(preds[-1])]
    
    return word

In [ ]:
predict(['we', 'are', 'to', 'study'])

In [ ]:
def accuracy():
    wrong = 0

    for context, target in data:
        if(predict(context) != target):
            wrong += 1
            
    return (1 - (wrong / len(data)))